In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

COVID = pd.read_excel('./코로나바이러스감염증-19_확진환자_발생현황_220501.xlsx', header=4)
COVID.drop(0,inplace=True)
COVID.replace({'국내발생(명)':'-'},{'국내발생(명)':'0'}, inplace=True)
COVID.replace({'해외유입(명)':'-'},{'해외유입(명)':'0'}, inplace=True)
COVID.replace({'사망(명)':'-'},{'사망(명)':'0'}, inplace=True)
COVID['일자'] = pd.to_datetime(COVID['일자'])
COVID = COVID.astype({'국내발생(명)':'int'})
COVID = COVID.astype({'해외유입(명)':'int'})
COVID = COVID.astype({'사망(명)':'int'})

In [48]:
C = pd.read_excel('./코로나바이러스감염증-19_확진환자_발생현황_220501.xlsx', sheet_name='연령별(10세단위)', header=4)
C.drop(0,inplace=True)
C = C.drop(['일자'], axis=1)

C.loc[C['0-9세'] == '-', '0-9세'] = 0
C.loc[C['10-19세'] == '-', '10-19세'] = 0
C.loc[C['20-29세'] == '-', '20-29세'] = 0
C.loc[C['30-39세'] == '-', '30-39세'] = 0
C.loc[C['40-49세'] == '-', '40-49세'] = 0
C.loc[C['50-59세'] == '-', '50-59세'] = 0
C.loc[C['60-69세'] == '-', '60-69세'] = 0
C.loc[C['70-79세'] == '-', '70-79세'] = 0
C.loc[C['80세이상'] == '-', '80세이상'] = 0
C = C.astype('int')
C = C.drop(['계(명)'], axis=1)

In [49]:
S = pd.read_excel('./코로나바이러스감염증-19_확진환자_발생현황_220501.xlsx', sheet_name='성별(남+여)', header=4)
S = S.drop(0)
S.drop(['일자','계(명)'], axis = 1, inplace=True)
S.replace({'남성(명)':'-'},{'남성(명)':'0'}, inplace=True)
S.replace({'여성(명)':'-'},{'여성(명)':'0'}, inplace=True)
S = S.astype('int')

A = pd.concat([C,S],axis=1)

In [50]:
X = A
y = COVID['사망(명)']

In [71]:
X

,0-9세,10-19세,20-29세,30-39세,40-49세,50-59세,60-69세,70-79세,80세이상,남성(명),여성(명)
1,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
829,7066,7162,10967,11328,11363,10628,9407,5268,3586,35133,41642
830,5183,5416,8776,8730,8825,7788,6817,3708,2217,26162,31298
831,4525,4751,7676,7359,7495,7158,6199,3438,1967,23235,27333
832,3738,4123,6243,5959,6150,6204,5713,3238,1918,19746,23540


In [51]:
lm = sm.OLS(y*2, X).fit()

In [81]:
p = (lm.predict(X)/2).round(0)

In [90]:
p_dead = pd.DataFrame({'추정사망(명)':p})
p_dead[p_dead['추정사망(명)']<=0] = 0

In [94]:
A

,0-9세,10-19세,20-29세,30-39세,40-49세,50-59세,60-69세,70-79세,80세이상,남성(명),여성(명)
1,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
829,7066,7162,10967,11328,11363,10628,9407,5268,3586,35133,41642
830,5183,5416,8776,8730,8825,7788,6817,3708,2217,26162,31298
831,4525,4751,7676,7359,7495,7158,6199,3438,1967,23235,27333
832,3738,4123,6243,5959,6150,6204,5713,3238,1918,19746,23540


In [108]:
predict_COVID = pd.concat([COVID['일자'],A,COVID['계(명)'],y,p_dead], axis=1)
predict_COVID.set_index('일자', inplace=True)
predict_COVID

,0-9세,10-19세,20-29세,30-39세,40-49세,50-59세,60-69세,70-79세,80세이상,남성(명),여성(명),계(명),사망(명),추정사망(명)
일자,,,,,,,,,,,,,,
2020-01-20,0,0,0,1,0,0,0,0,0,0,1,1,0,0.0
2020-01-21,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-22,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-24,0,0,0,0,0,1,0,0,0,1,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-27,7066,7162,10967,11328,11363,10628,9407,5268,3586,35133,41642,76775,141,129.0
2022-04-28,5183,5416,8776,8730,8825,7788,6817,3708,2217,26162,31298,57460,122,101.0
2022-04-29,4525,4751,7676,7359,7495,7158,6199,3438,1967,23235,27333,50568,136,83.0


In [109]:
predict_COVID.to_csv('./PJT_1.csv',encoding='utf-8-sig')